## Кривоногов Н.В., INN, практическое задание № 4

1. Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает. 
2. Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from keras.layers import Dropout
Dropout(rate=0.5)

In [3]:
from keras.layers import BatchNormalization

Sequential([
    BatchNormalization(input_shape=(8, 8, 2))
]).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 8, 8, 2)          8         
 ormalization)                                                   
                                                                 
Total params: 8
Trainable params: 4
Non-trainable params: 4
_________________________________________________________________


In [4]:
# установка параметров нейросети

batch_size = 512
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

170498071/170498071 [==============================] - 6s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [6]:
data_augmentation = True

In [7]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 42s 307ms/step - loss: 2.3043 - accuracy: 0.1034 - val_loss: 2.3002 - val_accuracy: 0.1311
Epoch 2/5
98/98 [==============================] - 28s 289ms/step - loss: 2.2985 - accuracy: 0.1163 - val_loss: 2.2942 - val_accuracy: 0.1325
Epoch 3/5
98/98 [==============================] - 29s 291ms/step - loss: 2.2901 - accuracy: 0.1268 - val_loss: 2.2801 - val_accuracy: 0.1383
Epoch 4/5
98/98 [==============================] - 28s 285ms/step - loss: 2.2659 - accuracy: 0.1467 - val_loss: 2.2379 - val_accuracy: 0.1786
Epoch 5/5
98/98 [==============================] - 28s 289ms/step - loss: 2.2055 - accuracy: 0.1832 - val_loss: 2.1366 - val_accuracy: 0.2425


In [8]:
# # сохранение модели и весов
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
    
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print(f'сохранить обученную модель как {model_path}')

# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 2.1366 - accuracy: 0.2425
Test loss: 2.1365511417388916
Test accuracy: 0.24250000715255737


#### Точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке, равна 0.2425.

### Попробую изменить Droput:

In [9]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [10]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 2.0190 - accuracy: 0.2743
Test loss: 2.019049882888794
Test accuracy: 0.2743000090122223


#### Точность распознавания образов cifar 10 сверточной нейронной сетью при изменении Dropout в каждом слое равна 0.2743 - улучшение.  

### Попробую изменить некоторые параметры data augmentation:

In [11]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_12 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [12]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-05, 
        rotation_range=5, 
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0., 
        zoom_range=0.15, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 30s 295ms/step - loss: 2.3044 - accuracy: 0.1067 - val_loss: 2.2938 - val_accuracy: 0.1091
Epoch 2/5
98/98 [==============================] - 29s 291ms/step - loss: 2.2899 - accuracy: 0.1194 - val_loss: 2.2795 - val_accuracy: 0.1160
Epoch 3/5
98/98 [==============================] - 28s 281ms/step - loss: 2.2688 - accuracy: 0.1340 - val_loss: 2.2439 - val_accuracy: 0.1507
Epoch 4/5
98/98 [==============================] - 28s 284ms/step - loss: 2.2251 - accuracy: 0.1644 - val_loss: 2.1788 - val_accuracy: 0.2048
Epoch 5/5
98/98 [==============================] - 29s 290ms/step - loss: 2.1631 - accuracy: 0.1983 - val_loss: 2.0968 - val_accuracy: 0.2284


In [13]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 2.0968 - accuracy: 0.2284
Test loss: 2.096782684326172
Test accuracy: 0.22840000689029694


#### Точность распознавания образов cifar 10 сверточной нейронной сетью при изменении некоторых параметров data augmentation равна 0.2284 - ухудшение.

### Попробую увеличить ширину сети (больше фильтров):

In [14]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 activation_18 (Activation)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 64)        36928     
                                                                 
 activation_19 (Activation)  (None, 30, 30, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 15, 15, 64)        0         
                                                      

In [15]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 35s 322ms/step - loss: 2.3018 - accuracy: 0.1054 - val_loss: 2.2963 - val_accuracy: 0.1452
Epoch 2/5
98/98 [==============================] - 30s 305ms/step - loss: 2.2930 - accuracy: 0.1269 - val_loss: 2.2832 - val_accuracy: 0.1647
Epoch 3/5
98/98 [==============================] - 29s 295ms/step - loss: 2.2700 - accuracy: 0.1562 - val_loss: 2.2360 - val_accuracy: 0.1989
Epoch 4/5
98/98 [==============================] - 30s 304ms/step - loss: 2.1916 - accuracy: 0.1935 - val_loss: 2.0974 - val_accuracy: 0.2347
Epoch 5/5
98/98 [==============================] - 30s 303ms/step - loss: 2.1149 - accuracy: 0.2078 - val_loss: 2.0446 - val_accuracy: 0.2516


In [16]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.0446 - accuracy: 0.2516
Test loss: 2.0446438789367676
Test accuracy: 0.2515999972820282


#### Точность распознавания образов cifar 10 сверточной нейронной сетью при увеличении ширины сети (больше фильтров) равна 0.2516 - улучшение.

### Попробую увеличить глубину сети (больше слоев):

In [17]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_24 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [18]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 30s 294ms/step - loss: 2.3016 - accuracy: 0.1005 - val_loss: 2.3008 - val_accuracy: 0.0994
Epoch 2/5
98/98 [==============================] - 29s 295ms/step - loss: 2.2993 - accuracy: 0.1056 - val_loss: 2.2990 - val_accuracy: 0.1253
Epoch 3/5
98/98 [==============================] - 28s 285ms/step - loss: 2.2971 - accuracy: 0.1122 - val_loss: 2.2963 - val_accuracy: 0.1436
Epoch 4/5
98/98 [==============================] - 28s 290ms/step - loss: 2.2934 - accuracy: 0.1177 - val_loss: 2.2915 - val_accuracy: 0.1628
Epoch 5/5
98/98 [==============================] - 29s 299ms/step - loss: 2.2858 - accuracy: 0.1272 - val_loss: 2.2817 - val_accuracy: 0.1741


In [19]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.2817 - accuracy: 0.1741
Test loss: 2.281749725341797
Test accuracy: 0.17409999668598175


#### Точность распознавания образов cifar 10 сверточной нейронной сетью при увеличении глубины сети (больше слоев) равна 0.1741 - резкое ухудшение.

### Попробую отключить data augmentation:

In [20]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_32 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_33 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 dropout_17 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [21]:
data_augmentation = False

In [22]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Не используется data augmentation
Epoch 1/5
98/98 [==============================] - 6s 49ms/step - loss: 2.3031 - accuracy: 0.1080 - val_loss: 2.2960 - val_accuracy: 0.1782
Epoch 2/5
98/98 [==============================] - 4s 40ms/step - loss: 2.2920 - accuracy: 0.1352 - val_loss: 2.2800 - val_accuracy: 0.2107
Epoch 3/5
98/98 [==============================] - 4s 40ms/step - loss: 2.2639 - accuracy: 0.1630 - val_loss: 2.2201 - val_accuracy: 0.2211
Epoch 4/5
98/98 [==============================] - 4s 38ms/step - loss: 2.1755 - accuracy: 0.1879 - val_loss: 2.0932 - val_accuracy: 0.2332
Epoch 5/5
98/98 [==============================] - 4s 41ms/step - loss: 2.1077 - accuracy: 0.2101 - val_loss: 2.0344 - val_accuracy: 0.2685


In [23]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 2.0344 - accuracy: 0.2685
Test loss: 2.034400224685669
Test accuracy: 0.2685000002384186


#### Точность распознавания образов cifar 10 сверточной нейронной сетью при отключении data augmentation равна 0.2685 - улучшение.

### Изменение таких параметров, как: Dropout, число фильтров, число слоев, изменение самих параметров в data augmentation существенно влияют на точность распознавания образов cifar 10 сверточной нейронной сетью. 
### Если говорить конкретно про рассмотренный пример, то подбор Dropout в слоях со значениями 0.15, 0.15, 0.25; увеличение числа фильтров до 64 и 128 позволили улучшить точность нейросети. Отказ от аугментации также способствовал улучшению точности нейронной сети. 
### А вот увеличение числа слоев, ответственных за свертку и max-pooling, до 3; установление значений zca_epsilon=1e-05, rotation_range=5, width_shift_range=0.15, height_shift_range=0.15, zoom_range=0.15 в data augmentation ухудшило её точность. 

#### Если бы нейросети нужно было работать с MNIST, CIFAR100 и IMAGENET, то в нее необходимо было бы внести определенные изменения, ведь:
- изображения в MNIST состоят из черно-белых пикселей, а не из цветных, как в CIFAR10; 
- в CIFAR100 на выходе 100 классов, сгруппированных в 20 суперклассов, а не 10, как в CIFAR10; 
- в IMAGENET 21841 категорий (по состоянию на август 2017 года) и многое другое. 

#### Эти датасеты отличаются количеством классов, размером картинок и их цветностью, величиной тренировочной и тестовой выборки, а также характеристиками разметки. Все эти отличия должны быть отражены при построении моделей нейронной сети для каждого конкретного датасета.